In [9]:
import numpy as np
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
#load data
file = open('f.txt').read()

In [14]:
#tokenisation
#standardisation
def tokenize_words(input):
    input= input.lower()
    tokenizer=RegexpTokenizer(r'\w+')
    tokens= tokenizer.tokenize(input)
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return "".join(filtered)

processed_inputs = tokenize_words(file)


In [26]:
#chars to nos
chars= sorted(list(set(processed_inputs)))
char_to_num= dict((c,i) for i, c in enumerate(chars))

In [27]:
#check if has workedd?\
input_len = len(processed_inputs)
vocab_len= len(chars)
print('total no of chars:', input_len)
print('total vocab:', vocab_len)

total no of chars: 232972
total vocab: 37


In [28]:
#seq lenght
seq_len = 100
x_data= []
y_data =[]


In [29]:
#loop seq
for i in range (0, input_len - seq_len,1):
    in_seq = processed_inputs[i:i + seq_len]
    out_seq = processed_inputs[i+seq_len]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
    
n_patterns = len(x_data)
print('total pat:', n_patterns)
    

total pat: 232872


In [31]:
 #conver in to np array
    
X= np.reshape(x_data, (n_patterns, seq_len, 1))
X = X/float(vocab_len)

In [32]:
#one ht encode

y= np_utils.to_categorical(y_data)

In [34]:
#create the modeld

model = Sequential()
model.add(LSTM(256, input_shape= (X.shape[1], X.shape[2]), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(256,return_sequences= True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation = 'softmax'))

In [36]:
#compile the kdoel
model.compile(loss = 'categorical_crossentropy', optimizer='adam')

In [38]:
#saving_weights

filepath= 'model_wrights_saved.hdf5'

checkpoint= ModelCheckpoint(filepath, monitor='loss', verbose =1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [39]:
#firmodel

model.fit(X,y, epochs=4, batch_size = 256, callbacks=desired_callbacks)

Epoch 1/4
232872/232872 [==============================] - 3436s 15ms/step - loss: 2.9288

Epoch 00001: loss improved from inf to 2.92880, saving model to model_wrights_saved.hdf5
Epoch 2/4
232872/232872 [==============================] - 3415s 15ms/step - loss: 2.9122

Epoch 00002: loss improved from 2.92880 to 2.91223, saving model to model_wrights_saved.hdf5
Epoch 3/4
232872/232872 [==============================] - 3434s 15ms/step - loss: 2.8843

Epoch 00003: loss improved from 2.91223 to 2.88428, saving model to model_wrights_saved.hdf5
Epoch 4/4
232872/232872 [==============================] - 3439s 15ms/step - loss: 2.8416

Epoch 00004: loss improved from 2.88428 to 2.84165, saving model to model_wrights_saved.hdf5


In [41]:
#recompile model with saved wts

filename= 'model_wrights_saved.hdf5'
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [42]:
#output of the model back into chars

num_to_char= dict((i,c) for i,c in enumerate(chars))

In [47]:
#random seed to jhelp gen

start=  np.random.randint(0,len(x_data)-1)
pattern= x_data[start]
print('Random Seed:')
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" joysightawfulmajesticnatureindeedalwayseffectsolemnizingmindcausingforgetpassingcareslifedeterminedg "


In [53]:
#gen the text

for i in range(1000):
    x= np.reshape(pattern, (1,len(pattern), 1))
    x = x/float(vocab_len)
    prediction = model.predict(x,verbose=0)
    index= np.argmax(prediction)
    result = num_to_char[index]
    seq_in =[num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern= pattern[1:len(pattern)]

eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee